<a href="https://colab.research.google.com/github/Serhii-Kravchenko-2022/Home_work_Data_Science8/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Home work for module 7 by
кравченко Сергій

# Рекомендаційні системи

# Частина перша

Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.

In [6]:
from google.colab import drive

drive.mount('/DataScience8')

Mounted at /DataScience8


In [9]:
file_ratings = '/DataScience8/MyDrive/DataScience8/ml-latest-small/ratings.csv'
file_movies = '/DataScience8/MyDrive/DataScience8/ml-latest-small/movies.csv'
file_tags = '/DataScience8/MyDrive/DataScience8/ml-latest-small/tags.csv'

ratings = pd.read_csv(file_ratings)
movies = pd.read_csv(file_movies)
tags = pd.read_csv(file_tags)

In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [34]:
!pip install scikit-surprise
import pandas as pd
import numpy as np
from surprise import SVD, NMF, SVDpp, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
merge_data = pd.merge(ratings, movies, on='movieId')
merge_data = pd.merge(merge_data, tags, on=['userId', 'movieId'])

In [20]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merge_data[['userId', 'movieId', 'rating']], reader)

In [31]:
result_list = []
algorithms = [SVD(), SVDpp(), NMF()]

for algorithm in algorithms:
    results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = pd.concat([pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']), tmp])
    result_list.append(tmp)

In [32]:
surprise_results = pd.DataFrame(result_list).set_index('Algorithm').sort_values('test_rmse')
surprise_results

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVDpp,0.533361,0.308001,3.421340,0.196427
SVD,0.560877,0.345614,0.114857,0.016376
NMF,0.581130,0.293820,0.105762,0.025147


Висновок:
Мінімальна середня квадратична помилка передбачення (RMSE) складає 0.533361 для моделі SVDpp Це свідчить про те, що модель має низьку помилку у передбаченні.

Мінімальна середня абсолютна помилка передбачення (MAE)  становить 0.29382 для моделі NMF. Це свідчить про високу точність моделі у передбаченні.

Мінімальний час навчання складає приблизно 0.105 секунди для моделі NMF, а час тестування - 0.016 секунди для моделі SVD.

Отже, оптимальною для великих датасетів є модель SVD. Якщо потрібна точність - модель SVDpp

In [35]:
trainset, testset = train_test_split(data, test_size=0.25)
for algorithm in algorithms:
    algorithm.fit(trainset)
    predictions = algorithm.test(testset)

    print(algorithm, accuracy.rmse(predictions))

RMSE: 0.5883
<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7f3a652ad270> 0.5883302299845197
RMSE: 0.5562
<surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7f3a652ad0c0> 0.5561967972172076
RMSE: 0.5770
<surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7f3a652aee90> 0.5770067205253285


# Друга частина
Для більшого заглиблення в роботу алгоритму, пропонуємо реалізувати алгоритм колабораційної фільтрації з нуля. Для цього ми можемо скористатись нашою домашньою роботою з 3-ого модуля. Якщо ми модифікуємо функцію втрат та розрахунок градієнтів, то зможемо побудувати алгоритм матричної факторизації.

In [36]:
from scipy import optimize
from scipy.io import loadmat

In [37]:
movie_ids = '/DataScience8/MyDrive/DataScience8/ml-latest-small/movie_ids.txt'
movies_mat = '/DataScience8/MyDrive/DataScience8/ml-latest-small/movies.mat'

In [38]:
def normalizeRatings(Y, R):
    m, n = Y.shape
    Ymean = np.zeros(m)
    Ynorm = np.zeros(Y.shape)

    for i in range(m):
        idx = R[i, :] == 1
        Ymean[i] = np.mean(Y[i, idx])
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]

    return Ynorm, Ymean


def loadMovieList():
    # Read the fixed movieulary list
    with open(movie_ids,  encoding='ISO-8859-1') as fid:
        movies = fid.readlines()

    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(' '.join(parts[1:]).strip())
    return movieNames


def computeNumericalGradient(J, theta, e=1e-4):
    numgrad = np.zeros(theta.shape)
    perturb = np.diag(e * np.ones(theta.shape))
    for i in range(theta.size):
        loss1, _ = J(theta - perturb[:, i])
        loss2, _ = J(theta + perturb[:, i])
        numgrad[i] = (loss2 - loss1)/(2*e)
    return numgrad


def checkCostFunction(cofiCostFunc, lambda_=0.):
    # Create small problem
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    # Zap out most entries
    Y = np.dot(X_t, Theta_t.T)
    Y[np.random.rand(*Y.shape) > 0.5] = 0
    R = np.zeros(Y.shape)
    R[Y != 0] = 1

    # Run Gradient Checking
    X = np.random.randn(*X_t.shape)
    Theta = np.random.randn(*Theta_t.shape)
    num_movies, num_users = Y.shape
    num_features = Theta_t.shape[1]

    params = np.concatenate([X.ravel(), Theta.ravel()])
    numgrad = computeNumericalGradient(
        lambda x: cofiCostFunc(x, Y, R, num_users, num_movies, num_features, lambda_), params)

    cost, grad = cofiCostFunc(params, Y, R, num_users,num_movies, num_features, lambda_)

    print(np.stack([numgrad, grad], axis=1))
    print('\nThe above two columns you get should be very similar.'
          '(Left-Your Numerical Gradient, Right-Analytical Gradient)')

    diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)
    print('If your cost function implementation is correct, then '
          'the relative difference will be small (less than 1e-9).')
    print('\nRelative Difference: %g' % diff)

In [39]:
names = loadMovieList()
len(names)

1682

In [41]:
data = loadmat(movies_mat)
Y, R = data['Y'], data['R']

print('Average rating for movie 1000 (',names[1000] ,'): %f / 5' %
      np.mean(Y[180, R[1000, :]]))

Average rating for movie 1000 ( Stupids, The (1996) ): 4.909862 / 5


In [42]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):
    # Unfold the U and W matrices from params
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    # You need to return the following values correctly
    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    J = (1 / 2) * np.sum(np.square((X.dot(Theta.T) - Y) * R)) + (lambda_ / 2) * np.sum(np.square(X)) + \
                                                                (lambda_ / 2) * np.sum(np.square(Theta))

    for i in range(R.shape[0]):

        idx = np.where(R[i, :] == 1)[0]
        Theta_temp = Theta[idx, :]
        Y_temp = Y[i, idx]
        X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]

    for j in range(R.shape[1]):

        idx = np.where(R[:, j] == 1)[0]
        X_temp = X[idx, :]
        Y_temp = Y[idx, j]
        Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]

    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return J, grad

In [43]:
checkCostFunction(cofiCostFunc, 1.5)

[[  8.25769069   8.25769069]
 [  2.20585765   2.20585765]
 [  2.58723598   2.58723598]
 [ 10.33266254  10.33266254]
 [ 15.98458484  15.98458484]
 [-22.83304154 -22.83304154]
 [  3.20343957   3.20343957]
 [  2.73941409   2.73941409]
 [  4.011        4.011     ]
 [  3.42518653   3.42518653]
 [ -8.83198215  -8.83198215]
 [ 14.69155982  14.69155982]
 [  5.32800546   5.32800546]
 [ -0.81523775  -0.81523775]
 [  1.12354476   1.12354476]
 [  0.60598562   0.60598562]
 [  1.51176171   1.51176171]
 [ -4.07899828  -4.07899828]
 [  6.03862074   6.03862074]
 [ 11.48226996  11.48226996]
 [-18.00920929 -18.00920929]
 [ -8.81090931  -8.81090931]
 [ -7.37787716  -7.37787716]
 [  8.71234347   8.71234347]
 [-14.3318822  -14.3318822 ]
 [ -8.78981177  -8.78981177]
 [ -6.04365208  -6.04365208]]

The above two columns you get should be very similar.(Left-Your Numerical Gradient, Right-Analytical Gradient)
If your cost function implementation is correct, then the relative difference will be small (less than 1

In [47]:
movieList = loadMovieList()

#  Before we will train the collaborative filtering model, we will first
#  add ratings that correspond to a new user that we just observed. This
#  part of the code will also allow you to put in your own ratings for the
#  movies in our dataset!
n_m = len(movieList)
#  Initialize my ratings
my_ratings = np.zeros(n_m)
# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[2] = 5
my_ratings[77] = 4
my_ratings[50] = 5
my_ratings[59]= 0
my_ratings[67] = 5
my_ratings[63] = 5
my_ratings[145] = 5
my_ratings[190] = 2
my_ratings[182] = 4
my_ratings[225] = 3
my_ratings[283] = 1
my_ratings[394] = 3
my_ratings[479] = 1
my_ratings[647] = 3

In [48]:
#  Add our own ratings to the data matrix
Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

#  Normalize Ratings
Ynorm, Ymean = normalizeRatings(Y, R)

#  Useful Values
num_movies, num_users = Y.shape
num_features = 7

# Set Initial Parameters (Theta, X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Set options for scipy.optimize.minimize
options = {'maxiter': 100}

# Set Regularization
lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

# Unfold the returned theta back into U and W
X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

<ipython-input-48-74526c60ba28>:23: DeprecationWarning: 'maxiter' has been deprecated in favor of 'maxfun' and will be removed in SciPy 1.11.0.
  res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,


In [49]:
# Make recommendations by computing the predictions matrix
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

movieList = loadMovieList()

ix = np.argsort(my_predictions)[::-1]

print('Top recommendations for you:')
print('----------------------------')
for i in range(15):
    j = ix[i]
    print('Predicting rating %.1f for movie %s' % (my_predictions[j], movieList[j]))

print('\nOriginal ratings provided:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d for %s' % (my_ratings[i], movieList[i]))

Top recommendations for you:
----------------------------
Predicting rating 5.0 for movie Saint of Fort Washington, The (1993)
Predicting rating 5.0 for movie Entertaining Angels: The Dorothy Day Story (1996)
Predicting rating 5.0 for movie Aiqing wansui (1994)
Predicting rating 5.0 for movie Marlene Dietrich: Shadow and Light (1996)
Predicting rating 5.0 for movie Santa with Muscles (1996)
Predicting rating 5.0 for movie They Made Me a Criminal (1939)
Predicting rating 5.0 for movie Prefontaine (1997)
Predicting rating 5.0 for movie Star Kid (1997)
Predicting rating 5.0 for movie Great Day in Harlem, A (1994)
Predicting rating 5.0 for movie Someone Else's America (1995)
Predicting rating 4.6 for movie Pather Panchali (1955)
Predicting rating 4.6 for movie Close Shave, A (1995)
Predicting rating 4.5 for movie Wrong Trousers, The (1993)
Predicting rating 4.5 for movie Some Mother's Son (1996)
Predicting rating 4.5 for movie Maya Lin: A Strong Clear Vision (1994)

Original ratings provid